In [1]:
import pandas as pd
import praw
import numpy as np
from nltk import word_tokenize
import matplotlib
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

In [2]:
posts = pd.read_csv("/Users/alliesaizan/Downloads/data.csv")
posts.shape

(114899, 6)

In [ ]:
# Drop rows with NA text
posts = posts.loc[(posts["selftext"].isna() == False)]

posts.shape

In [ ]:
# Creating some features to help with data cleaning
posts["tokenized"] = posts["selftext"].apply(word_tokenize)
posts["text_len"] = posts["tokenized"].apply(len)

## The bulk of the posts are less than 2000 words long.

In [ ]:
num_bins = 50

fig, ax = plt.subplots()

n, bins, patches = ax.hist(posts["text_len"].tolist(), num_bins, density=1)

plt.show()

What does the histogram look like at a more granular level (with # words less than 2,000)?

In [ ]:
fig, ax = plt.subplots()
n, bins, patches = ax.hist(posts.loc[posts.text_len < 2000, "text_len"].tolist(), num_bins, density=1)

plt.show() # lotsa zeroes

What is the exact highest point of the histogram?

In [ ]:
bin_max = np.where(n == n.max())
bins[bin_max][0]

At the 600 word mark, the text appears to be legitimate stories and not callouts or questions. But looking at posts greater than 600 words only leaves us with 67% of the data. At the 250 word mark, it was looking like mosts of the posts were creative works. One option is to hand-classify posts with lengths between 250 and 600 and train a classifier on the tagged data.

In [ ]:
posts.loc[posts.text_len == 600]
posts.loc[posts.text_len > 600].shape

In [ ]:
# Arbitrary 250-word cutoff, because hand classification sounds like too much damn work
posts_subset = posts.loc[posts.text_len >= 250]
posts_subset.shape

In [ ]:
# Export a sample of the data that is less than 10mb. 
posts_subset = posts_subset.sample(n=8000)

In [ ]:
posts_txt = "".join(["|startoftext|" + i + "|endoftext|" for i in posts_subset.selftext.to_list()])

with open("posts.txt", "a") as f:
    f.writelines(posts_txt)
f.close()

In [ ]:
pickle.dump(posts_txt, open("posts.pkl", "wb"))